<a href="https://colab.research.google.com/github/ygh1025/demo-repo/blob/main/tensorflow_class2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"


--2022-05-01 02:39:48--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.72.44.106, 2600:1417:3f:128b::e59, 2600:1417:3f:1285::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.72.44.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   212MB/s    in 3.9s    

2022-05-01 02:39:52 (200 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [3]:
import zipfile
import os


In [4]:
from shutil import copyfile

In [5]:
fileref=zipfile.ZipFile('/tmp/cats-and-dogs.zip','r')
fileref.extractall()
fileref.close()

In [6]:
import random

In [7]:
os.listdir('PetImages/Cat')[0].size

AttributeError: ignored

In [ ]:
os.join

In [8]:
os.mkdir('train')
os.mkdir('test')
os.mkdir('train/Cat')
os.mkdir('train/Dog')
os.mkdir('test/Cat')
os.mkdir('test/Dog')

FileExistsError: ignored

In [51]:
def files_copy(source,destination,file_size=0.9):
  filenames=os.listdir(source)
  file=[]
  for filename in filenames:
    if os.path.getsize(os.path.join(source,filename))>0:
      file.append(filename)
  sfilename=random.sample(file,len(file))
  total_size=len(sfilename)
  final_size=int(total_size*file_size)
  final_filename=sfilename[:final_size]
  for name in final_filename:
    copyfile(os.path.join(source,name),os.path.join(destination,name))
  

In [52]:
files_copy('PetImages/Cat','train/Cat')

In [53]:
files_copy('PetImages/Dog','train/Dog')
files_copy('PetImages/Cat','test/Cat',file_size=0.1)
files_copy('PetImages/Dog','test/Dog',file_size=0.1)

In [9]:
print(len(os.listdir('train/Cat')))
print(len(os.listdir('train/Dog')))
print(len(os.listdir('test/Cat')))
print(len(os.listdir('test/Dog')))


11250
11250
1250
1250


In [31]:
from tensorflow.keras.layers import Conv2D,AveragePooling2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [11]:
train_Datagen=ImageDataGenerator(rescale=1./255.,rotation_range=40,width_shift_range=0.2,
                                 height_shift_range=0.2,zoom_range=0.2,shear_range=0.2,
                                 horizontal_flip=True,vertical_flip=True)
test_Datagen=ImageDataGenerator(rescale=1./255.)
train_data=train_Datagen.flow_from_directory('train/',target_size=(150,150),batch_size=100,class_mode='binary')
test_data=train_Datagen.flow_from_directory('test/',target_size=(150,150),batch_size=20,class_mode='binary')


Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [12]:
model=Sequential([
    Conv2D(32,kernel_size=(4,4),strides=2,activation='relu',input_shape=(150,150,3)),
    MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    Conv2D(64,kernel_size=(4,4),strides=2,activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    Conv2D(64,kernel_size=(4,4),strides=2,activation='relu'),
    MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1,activation='sigmoid')    
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 74, 74, 32)        1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 37, 37, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 64)        32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          65600     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 64)         0

In [ ]:
: 0.0698 - accuracy: 0.9775 - val_loss: 0.0631 - val_accuracy: 0.9820

In [13]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.003),metrics=['accuracy'])
hist=model.fit(train_data,epochs=50,steps_per_epoch=int(11250/100),validation_steps=int(1250/20),validation_data=test_data)

Epoch 1/50
112/112 [==============================] - 90s 766ms/step - loss: 0.7013 - accuracy: 0.5223 - val_loss: 0.6917 - val_accuracy: 0.5323
Epoch 2/50
 11/112 [=>............................] - ETA: 1:10 - loss: 0.6884 - accuracy: 0.5218

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

112/112 [==============================] - 86s 765ms/step - loss: 0.6886 - accuracy: 0.5454 - val_loss: 0.7429 - val_accuracy: 0.4935
Epoch 3/50
112/112 [==============================] - 87s 778ms/step - loss: 0.6882 - accuracy: 0.5604 - val_loss: 0.7059 - val_accuracy: 0.5371
Epoch 4/50
112/112 [==============================] - 86s 764ms/step - loss: 0.6802 - accuracy: 0.5900 - val_loss: 0.6465 - val_accuracy: 0.6339
Epoch 5/50
112/112 [==============================] - 86s 768ms/step - loss: 0.6629 - accuracy: 0.6019 - val_loss: 0.6486 - val_accuracy: 0.6347
Epoch 6/50
112/112 [==============================] - 86s 770ms/step - loss: 0.6510 - accuracy: 0.6221 - val_loss: 0.6182 - val_accuracy: 0.6653
Epoch 7/50
112/112 [==============================] - 84s 753ms/step - loss: 0.6578 - accuracy: 0.6306 - val_loss: 0.5983 - val_accuracy: 0.6911
Epoch 8/50
112/112 [==============================] - 84s 754ms/step - loss: 0.6438 - accuracy: 0.6263 - val_loss: 0.6235 - val_accuracy: 0.6

KeyboardInterrupt: ignored

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Sequential,Model


In [13]:
premodel=InceptionV3(input_shape=(150,150,3),include_top=False,weights=None)

In [14]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5



--2022-04-30 01:57:48--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.197.128, 173.194.74.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   187MB/s    in 0.4s    

2022-04-30 01:57:48 (187 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [15]:
local_weight='/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
premodel.load_weights(local_weight)
for layer in premodel.layers:
  layer.trainable=False
#premodel.summary()

In [16]:
last_layer=premodel.get_layer('mixed7')
last_output=last_layer.output

In [17]:
X=Flatten()(last_output)
X=Dense(512,activation='relu')(X)
X=Dense(1,activation='sigmoid')(X)
model=Model(inputs=premodel.input,outputs=X)
#model.summary()


In [21]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_data,epochs=20,validation_data=test_data)


Epoch 1/20
 18/225 [=>............................] - ETA: 2:56 - loss: 0.1433 - accuracy: 0.9406

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

225/225 [==============================] - 216s 940ms/step - loss: 0.1495 - accuracy: 0.9379 - val_loss: 0.1330 - val_accuracy: 0.9460
Epoch 2/20
225/225 [==============================] - 210s 933ms/step - loss: 0.1469 - accuracy: 0.9397 - val_loss: 0.1427 - val_accuracy: 0.9428
Epoch 3/20
225/225 [==============================] - 205s 912ms/step - loss: 0.1455 - accuracy: 0.9402 - val_loss: 0.1305 - val_accuracy: 0.9488
Epoch 4/20
225/225 [==============================] - 206s 917ms/step - loss: 0.1411 - accuracy: 0.9415 - val_loss: 0.1459 - val_accuracy: 0.9428
Epoch 5/20
225/225 [==============================] - 204s 905ms/step - loss: 0.1405 - accuracy: 0.9418 - val_loss: 0.1330 - val_accuracy: 0.9468
Epoch 6/20
225/225 [==============================] - 205s 911ms/step - loss: 0.1397 - accuracy: 0.9426 - val_loss: 0.1358 - val_accuracy: 0.9424
Epoch 7/20
225/225 [==============================] - 207s 920ms/step - loss: 0.1364 - accuracy: 0.9443 - val_loss: 0.1326 - val_accura

In [1]:
!gdown --id 1z0DkA9BytlLxO1C0BAWzknLyQmZAp0HR

!gdown --id 1z1BIj4qmri59GWBG4ivMNFtpZ4AXIbzg

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1z0DkA9BytlLxO1C0BAWzknLyQmZAp0HR
To: /content/sign_mnist_train.csv
100% 83.3M/83.3M [00:00<00:00, 85.6MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1z1BIj4qmri59GWBG4ivMNFtpZ4AXIbzg
To: /content/sign_mnist_test.csv
100% 21.8M/21.8M [00:00<00:00, 58.0MB/s]


In [3]:
ls

sample_data/


In [2]:
import csv
import numpy as np

In [3]:
def csv_file(source):
  with open(source,'r') as file:
    lines=csv.reader(file,delimiter=',')
    next(lines)
    label=[]
    data=[]
    for line in lines:
      label.append(line[0])
      data.append(line[1:])
  return np.array(data,dtype='float32'),np.array(label,dtype='float32')

In [4]:
train_X,train_y=csv_file('/content/sign_mnist_train.csv')

In [5]:
test_X,test_y=csv_file('/content/sign_mnist_test.csv')

In [6]:
train_X=train_X.reshape(-1,28,28,1)
test_X=test_X.reshape(-1,28,28,1)

In [19]:
train_X[0]

array([107., 118., 127., 134., 139., 143., 146., 150., 153., 156., 158.,
       160., 163., 165., 159., 166., 168., 170., 170., 171., 171., 171.,
       172., 171., 171., 170., 170., 169., 111., 121., 129., 135., 141.,
       144., 148., 151., 154., 157., 160., 163., 164., 170., 119., 152.,
       171., 171., 170., 171., 172., 172., 172., 172., 172., 171., 171.,
       170., 113., 123., 131., 137., 142., 145., 150., 152., 155., 158.,
       161., 163., 164., 172., 105., 142., 170., 171., 171., 171., 172.,
       172., 173., 173., 172., 171., 171., 171., 116., 125., 133., 139.,
       143., 146., 151., 153., 156., 159., 162., 163., 167., 167.,  95.,
       144., 171., 172., 172., 172., 172., 172., 173., 173., 173., 172.,
       172., 171., 117., 126., 134., 140., 145., 149., 153., 156., 158.,
       161., 163., 164., 175., 156.,  87., 154., 172., 173., 173., 173.,
       173., 173., 174., 174., 174., 173., 172., 172., 119., 128., 136.,
       142., 146., 150., 153., 156., 159., 163., 16

In [21]:
np.unique(train_y)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.],
      dtype=float32)

In [34]:
train_dataGen=ImageDataGenerator(rescale=1./255.,rotation_range=0.4,width_shift_range=0.2,
                                 height_shift_range=0.2,zoom_range=0.2,shear_range=0.2,vertical_flip=True,
                                 horizontal_flip=True)
test_dataGen=ImageDataGenerator(rescale=1/255.)
train_data=train_dataGen.flow(train_X,train_y,batch_size=32)
test_data=test_dataGen.flow(test_X,test_y,batch_size=32)

In [42]:
model=Sequential([
      Conv2D(64,kernel_size=3,strides=1,activation='relu',input_shape=(28,28,1)),
      MaxPooling2D(2,2),
      Conv2D(64,kernel_size=3,strides=1,activation='relu'),
      MaxPooling2D(2,2),
      Flatten(),
      Dense(128,activation='relu'),
      Dense(26,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 1600)              0         
                                                                 
 dense_20 (Dense)            (None, 128)             

In [43]:
hist=model.fit(train_data,epochs=15,steps_per_epoch=len(train_X)/32,validation_steps=len(test_X)/32,validation_data=test_data)

Epoch 1/15
857/857 [==============================] - 11s 13ms/step - loss: 2.8528 - accuracy: 0.1336 - val_loss: 2.2585 - val_accuracy: 0.2450
Epoch 2/15
857/857 [==============================] - 11s 12ms/step - loss: 2.0922 - accuracy: 0.3291 - val_loss: 1.6950 - val_accuracy: 0.4459
Epoch 3/15
857/857 [==============================] - 11s 13ms/step - loss: 1.7573 - accuracy: 0.4257 - val_loss: 1.2384 - val_accuracy: 0.5697
Epoch 4/15
857/857 [==============================] - 11s 12ms/step - loss: 1.5420 - accuracy: 0.4920 - val_loss: 1.1388 - val_accuracy: 0.5986
Epoch 5/15
857/857 [==============================] - 11s 12ms/step - loss: 1.3622 - accuracy: 0.5535 - val_loss: 0.9160 - val_accuracy: 0.6899
Epoch 6/15
857/857 [==============================] - 11s 12ms/step - loss: 1.2236 - accuracy: 0.5930 - val_loss: 0.8494 - val_accuracy: 0.6852
Epoch 7/15
857/857 [==============================] - 11s 12ms/step - loss: 1.1139 - accuracy: 0.6276 - val_loss: 0.7740 - val_accuracy:

In [44]:
model.evaluate(test_X,test_y)

225/225 [==============================] - 1s 3ms/step - loss: 302.6133 - accuracy: 0.5567


[302.61334228515625, 0.5567484498023987]

In [14]:
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5

# Get the Horse or Human Validation dataset
!gdown --id 1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
To: /content/horse-or-human.zip
100% 150M/150M [00:02<00:00, 60.2MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U
To: /content/validation-horse-or-human.zip
100% 11.5M/11.5M [00:00<00:00, 21.7MB/s]


In [16]:
os.mkdir('hhtrain/')
os.mkdir('hhtest/')

In [17]:
zipref=zipfile.ZipFile('/content/horse-or-human.zip','r')
zipref.extractall('hhtrain/')
zipref.close()
zipref=zipfile.ZipFile('/content/validation-horse-or-human.zip','r')
zipref.extractall('hhtest/')
zipref.close()


In [22]:
len(os.listdir('hhtest/horses'))

128

In [23]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


--2022-05-01 03:14:26--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   116MB/s    in 0.7s    

2022-05-01 03:14:27 (116 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [24]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [25]:
pre_model=InceptionV3(input_shape=(150,150,3),weights=None,include_top=False)
weight=pre_model.load_weights('/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
for layer in pre_model.layers:
  layer.trainable=False
last_layer=pre_model.get_layer('mixed7')
last_output=last_layer.output

In [26]:
train_gen=ImageDataGenerator(rescale=1./255.)
test_gen=ImageDataGenerator(rescale=1./255.)
train_data=train_gen.flow_from_directory('hhtrain/',target_size=(150,150),batch_size=30,class_mode='binary')
test_data=test_gen.flow_from_directory('hhtest/',target_size=(150,150),batch_size=10,class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [28]:
import tensorflow as tf

In [29]:
class mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs=None):
    if logs.get('accuracy') is not None and logs.get('accuracy')>=0.999:
      print('the goal reaches 99.9% and has to stop!')
      self.model.stop_training=True

In [32]:
X=Flatten()(last_output)
X=Dense(128,activation='relu')(X)
X=Dense(1,activation='sigmoid')(X)
model=Model(inputs=pre_model.input,outputs=X)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [33]:
model.fit(train_data,epochs=20,validation_data=test_data,callbacks=[mycallback()])

Epoch 1/20
35/35 [==============================] - 12s 245ms/step - loss: 0.0400 - accuracy: 0.9796 - val_loss: 5.5239e-04 - val_accuracy: 1.0000
Epoch 2/20
35/35 [==============================] - 7s 192ms/step - loss: 1.6582e-04 - accuracy: 1.0000 - val_loss: 0.0843 - val_accuracy: 0.9922
